In [1]:
import pandas as pd
import numpy as np

In [2]:
sales_train = pd.read_csv('./sales_train.csv')
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [4]:
test = pd.read_csv('./test.csv')
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

### matrix에 월별 shop별, type_code별 평균 판매 수량(date_shop_type_avg_item_cnt) lag 추가하기
- 월 별, 가게 별, 대분류 별로 아이템 평균 판매 수량이다.

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']

# groupby로 묶여있던? 인덱스를 row 갯수에 맞춰서 풀어주는 코드
group.reset_index(inplace=True)
 
# matrix와 위 만든 컬럼을 merge하는 작업 
# 'date_block_num', 'shop_id', 'type_code'를 기준으로 left 방식으로 붙임
matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'type_code'], how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)

# 카테고리 별 이어서 1 개월 전의 데이터 컬럼들만 추가함
# 위 함수를 보면 date_block_num 을 1 증가 시켜서 이전 판매량을 이번 달에 추가
# 하기 위한 키로 사용 
# date_shop_type_avg_item_cnt_lag_1와 같은 방식으로 1개월 전의 데이터임을 표시
# 그리고 남은 데이터 제거
matrix = lag_feature(matrix, [1], 'date_shop_type_avg_item_cnt')
matrix.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)

### matrix에 월별 shop별, subtype_code별 평균 판매 수량(date_shop_subtype_avg_item_cnt) lag 추가하기
- 월 별, 가게 별, 중분류 혹은 소분류 별로 아이템 평균 판매 수량이다.
- 위 데이터와 다른 점은 'type_code' --> 'subtype_code'로 바뀐 점 뿐이다.

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix['date_shop_subtype_avg_item_cnt'] = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_subtype_avg_item_cnt')
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)